### Imports

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Internal Imports

In [11]:
# Find ".env" file and add the package to $PATH
import os, sys
import typing as t
from typing import Dict, TypeAlias, Any
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

# Use local package for modularity
import emotion_analysis as ea
from emotion_analysis import config
from emotion_analysis.data.dataset import ECACDataset
from emotion_analysis.model.trainer import TrainerModule
from emotion_analysis.data.loader import DefaultDataLoader
from emotion_analysis.model.pretrained import load_text_model
from emotion_analysis.model.model import EmotionCauseTextModel
from emotion_analysis.data.types import TrainSplit, DataSplit, SubTask
from emotion_analysis.data.transform import DataTokenize, DataTransform, DataCollator

External Imports

In [12]:
import jax
import jax.numpy as jnp
import jax.random as jrm
from jax import Array
import jax.tree_util as tree_util
from jax.tree_util import tree_structure
from jax.typing import ArrayLike, DTypeLike
import flax
import flax.linen as nn
import numpy as np
import optax as opt
import mlflow as mlf
from torch.utils.data import Dataset, DataLoader, Subset, random_split

### Dataset

In [13]:
# Data Preprocessors
text_encoder_pretrained = load_text_model(config.model_repo, config.cache_dir)
tokenizer = text_encoder_pretrained.tokenizer
tokenize = DataTokenize( tokenizer,  max_seq_len=config.max_uttr_len)
transform = DataTransform(tokenize, max_conv_len=config.max_conv_len)
collator = DataCollator(transform)

# Load data subsets
dataset: Dict[DataSplit, ECACDataset] = ECACDataset.read_data(config.data_dir, config.subtask)
ds_train, ds_valid, ds_test = *random_split(dataset['train'], [0.75, 0.25]), dataset['test']
num_classes: int = dataset['train'].num_emotions

# Train: drop_last=True to avoid JAX graph recompilation
dataloader: t.Dict[TrainSplit, DataLoader[t.Dict[str, Array]]] = {
    'train': DefaultDataLoader(ds_train,  shuffle=True, collate_fn=collator, drop_last=True),
    'valid': DefaultDataLoader(ds_train, shuffle=False, collate_fn=collator),
    'test' : DefaultDataLoader( ds_test, shuffle=False, collate_fn=collator),
}

### Model

In [14]:
key = jrm.PRNGKey(config.seed)
key, trainer_key = jrm.split(key, 2)

trainer = TrainerModule(
    key=trainer_key,
    finetune=config.finetune,
    batch_size=config.batch_size,
    max_conv_len=config.max_conv_len,
    max_uttr_len=config.max_uttr_len,
    text_model_repo=config.model_repo,
    learning_rate=config.learning_rate,
)

In [16]:
for i, X in enumerate(dataloader['train']):
    loss, key, trainer.state = trainer.train_step(key, trainer.state, X)
    print(i, len(dataloader['train']), loss)

ValueError: could not convert string to float: ''

In [7]:
value, grad

NameError: name 'value' is not defined

In [ ]:
def test_jit(x, params, train, drop_key):
    y= trainer.model.apply({ 'params': params }, **x, train=train, rngs={ 'dropout': drop_key })
    return y

In [ ]:
speed = jax.jit(test_jit, static_argnames='train')

In [ ]:
for X in dataloader['valid']:
    X = { 'input_ids': X['input_ids'], 'uttr_attn_mask': X['attention_mask'], 'conv_attn_mask': X['input_mask'] }
    y = speed(X, trainer.state.params, True, key)
    print('ok')

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [ ]:
# Train only the classifier for now...
should_freeze = lambda p, _: 'frozen' if 'text_encoder' in p else 'trainable'
param_labels = flax.traverse_util.path_aware_map(should_freeze, params)
tx = opt.multi_transform({ 'trainable': opt.adamw(2e-4), 'frozen': opt.set_to_zero() }, param_labels)
opt_state = tx.init(params)